<center> <h1>Algorithms in Computational Biology (INFO-F438)</h1> </center>

<center> <h2>Assignment 1: Optimal Protein Folding in the HP Model</h2> </center>
 
<center> <h3>Guillaume Buisson-Chavot</h3> </center> 
<center> <h3>matricule: 000465822 </h3> </center> 

In [31]:
import numpy as np
from datetime import datetime
import copy

In [32]:
def two_first_elements(n):
    """returns coordinates of the two first elements of a sequence 
    of length n placed to be sure that the space of all sequences of 
    lenght n will have positive coordinates
    """
    return [(n-2,n-4),(n-2,n-3)]

In [33]:
#tests
print(two_first_elements(9))

[(7, 5), (7, 6)]


In [34]:
def display(list_of_indexes, string_of_H_and_P):
    """builts and displays the grid (np.array) with the corresponding protein conformation from HP model
    """ 
    if len(list_of_indexes) == 0:
        print("list shouldn't be empty!")
        matrix = np.array((1,1),0)
    else:    
        l1, l2, new_list_of_indexes =[],[], []
        max_col, min_col, max_row, min_row = 0, 0, 0, 0
        n = len(string_of_H_and_P)
        if n >= 4:
            for i in range(n):
                l1.append(list_of_indexes[i][0])
                l2.append(list_of_indexes[i][1])
            max_row, min_row, max_col, min_col = max(l1), min(l1), max(l2), min(l2)           
            matrix = np.full((max_row-min_row+1, max_col-min_col+1),"          ")
            seq = list(enumerate(string_of_H_and_P, 1))
            for elem in list_of_indexes:
                new_list_of_indexes.append((elem[0]-min_row,elem[1]-min_col))   
            for i in range(n):
                matrix[new_list_of_indexes[i][0]][new_list_of_indexes[i][1]]= " "+ str(seq[i])+" "
        else:
            matrix = "Please give a sequence of minimum length = 4!"
    return matrix   

In [35]:
#tests
print(two_first_elements(9))
print(display([(7,5),(7,6),(6,6),(5,6),(5,7),(6,7),(6,8),(5,8),(4,8),(4,7)],'PPHPHHPHPH'))
print(display([(5,6),(5,7),(5,8)],'PPP'))

[(7, 5), (7, 6)]
[['          ' '          ' " (10, 'H')" " (9, 'P') "]
 ['          ' " (4, 'P') " " (5, 'H') " " (8, 'H') "]
 ['          ' " (3, 'H') " " (6, 'H') " " (7, 'P') "]
 [" (1, 'P') " " (2, 'P') " '          ' '          ']]
Please give a sequence of minimum length = 4!


In [36]:
def score(list_of_indexes, string, score = 0):
    """returns the score and the pairs between which there is a connection
    list_of_indexes: is a liste of tupples, each tupple representing an index
    string: is a the sequence of H and P
    """
    intermediate_liste = copy.copy(list_of_indexes)
    pairs_of_H = []
    for index in intermediate_liste:
        list_of_indexes_around = [(index[0],index[1]+1), (index[0]-1,index[1]), (index[0],index[1]-1), (index[0]+1,index[1])]
        if index == intermediate_liste[0]:
            list_of_indexes_around.remove(intermediate_liste[1])
            for i in range(len(list_of_indexes_around)):
                if string[intermediate_liste.index(index)] == 'H':
                    if list_of_indexes_around[i] in intermediate_liste and string[intermediate_liste.index(list_of_indexes_around[i])] == 'H':
                        pairs_of_H.append((index,list_of_indexes_around[i]))
        elif index == list_of_indexes[-1]:
            list_of_indexes_around.remove(intermediate_liste[-2])
            for i in range(len(list_of_indexes_around)):
                if string[intermediate_liste.index(index)] == 'H':
                    if list_of_indexes_around[i] in intermediate_liste and string[intermediate_liste.index(list_of_indexes_around[i])] == 'H':
                        pairs_of_H.append((index,list_of_indexes_around[i]))
        else:
            list_of_indexes_around.remove(intermediate_liste[intermediate_liste.index(index) - 1])
            list_of_indexes_around.remove(intermediate_liste[intermediate_liste.index(index) + 1])
            for i in range(len(list_of_indexes_around)):
                if string[intermediate_liste.index(index)] == 'H':
                    if list_of_indexes_around[i] in intermediate_liste and string[intermediate_liste.index(list_of_indexes_around[i])] == 'H':
                        pairs_of_H.append((index,list_of_indexes_around[i]))
    #print("pairs_of_H=", pairs_of_H)                    
    pairs = []
    while len(pairs_of_H)>1:
        elem = pairs_of_H[0]
        reverse_elem = ((elem[1],elem[0]))        
        if reverse_elem in pairs_of_H:
            pairs_of_H.remove(pairs_of_H[pairs_of_H.index(reverse_elem)])
            pairs_of_H.pop(0)
            score += 1
            pairs.append(elem)
    #print("-------- in score fonction --------")        
    #print("score=",score) 
    #print("pairs=",pairs)
    #print("-----------------------------------")
    return score, pairs



In [37]:
#tests
print(score([(7,5),(7,6),(6,6),(5,6),(5,7),(6,7),(6,8),(5,8),(4,8),(4,7)],'PPHPHHPHPH')) #score(HHPdebut)= 2 et score(PPHdebut)= 3

(3, [((6, 6), (6, 7)), ((5, 7), (5, 8)), ((5, 7), (4, 7))])


In [38]:
def checking(indexes_list, direction):  #direction = left or right or middle
    """returns the following index in function of the choosen direction and None if the index is already full
    """
    if direction == 'left' or direction == 'right' or direction == 'middle':
        if direction == 'left':
            a, b = 0, 0
            c, d = -1, 1
        elif direction == 'right':
            a, b = 0, 0
            c, d = 1, -1
        elif direction == 'middle':
            c, d = 0, 0
            a, b = 1, -1
        if indexes_list[-1][0] == indexes_list[-2][0] and indexes_list[-1][1] > indexes_list[-2][1]: #horiz/droite
            index_left = (indexes_list[-1][0]+c,indexes_list[-1][1]+a)
            if index_left in indexes_list:
                res = None
            else:
                res = index_left
        elif indexes_list[-1][0] == indexes_list[-2][0] and indexes_list[-1][1] < indexes_list[-2][1]: #horiz/gauche
            index_left = (indexes_list[-1][0]+d,indexes_list[-1][1]+b)
            if index_left in indexes_list:
                res = None
            else:
                res = index_left
        elif indexes_list[-1][1] == indexes_list[-2][1] and indexes_list[-1][0] > indexes_list[-2][0]: #vert/bas
            index_left = (indexes_list[-1][0]+a,indexes_list[-1][1]+d)
            if index_left in indexes_list:
                res = None
            else:
                res = index_left
        elif indexes_list[-1][1] == indexes_list[-2][1] and indexes_list[-1][0] < indexes_list[-2][0]:  #vert/haut  
            index_left = (indexes_list[-1][0]+b,indexes_list[-1][1]+c)
            if index_left in indexes_list:
                res = None
            else:
                res = index_left
    else:
        res = "please give a direction equal to left or right or middle!"
    return res    

In [39]:
#test
print(checking([(5,6),(5,7)], 'left'))
print(checking([(5,6),(5,7)], 'right'))
print(checking([(5,6),(5,7)], 'middle'))
print(checking([(5,7),(5,6)], 'left'))
print(checking([(5,7),(5,6)], 'right'))
print(checking([(5,7),(5,6)], 'middle'))

(4, 7)
(6, 7)
(5, 8)
(6, 6)
(4, 6)
(5, 5)


In [40]:
import os
string_of_H_and_P = "PPPPPPP"
total = 3**(len(string_of_H_and_P)-2)
truc= []


from IPython.display import clear_output

    
    
def branch_addition(l ,k ): #k = profondeur et n = hauteur ie profondeur max
    best_score_inter = 0 
    new_sol= []
    #print("recurrence=",l)
    if k == len(string_of_H_and_P):
        best_score_inter = score(l, string_of_H_and_P)[0]
        new_sol = l
        #print("new_sol=",new_sol)
        truc.append(1)
        clear_output(wait=True)
        print(truc.count(1),"/ (max = ",total, ")")
        
    else:
        for dir in ['left', 'middle' , 'right']:
            new_index = checking(l, dir) #return None if busy, index empty if empty
            if new_index != None:
                #print('inside2')
                l2 = copy.copy(l)
                l2.append(new_index)
                a,b = branch_addition(l2, k+1)
                if a >= best_score_inter:
                    best_score_inter = a
                    #print("best score_inter =", best_score_inter)
                    new_sol = b
                    #print("new sol=", b)
                    
    return best_score_inter, new_sol 

In [41]:
n = len(string_of_H_and_P)
list_of_two_first_elements = two_first_elements(n)
print('two first elements=',list_of_two_first_elements)
start = datetime.now()
scoring, sol = branch_addition(list_of_two_first_elements, 2)
end = datetime.now()
elapsed = end - start
formatted_computation_time = float('%i.%.6i' % (elapsed.seconds, elapsed.microseconds))
print('computation time =',formatted_computation_time, 'seconds')
print('string_of_H_and_P =', string_of_H_and_P)
print("lenght of the sequence=",n)
print("solution=", sol)
print("score=", scoring)
print(display(sol,string_of_H_and_P))

195 / (max =  243 )
computation time = 0.52417 seconds
string_of_H_and_P = PPPPPPP
lenght of the sequence= 7
solution= [(5, 3), (5, 4), (6, 4), (6, 3), (6, 2), (5, 2), (4, 2)]
score= 0
[[" (7, 'P') " '          ' '          ']
 [" (6, 'P') " " (1, 'P') " " (2, 'P') "]
 [" (5, 'P') " " (4, 'P') " " (3, 'P') "]]


In [42]:
#This function is written to prune the branch and bound tree
def are_symmetrical(seq1, seq2):
    """
    returns 1 if seq1 and seq2 are symmetrical else 0   
    """
    flag = 1
    for i in range(len(seq1)):
        if flag == 0:
            pass
        else:
            if seq1[i] == seq2[i]:
                flag = 1
            elif seq1[i][1] == seq1[i-1][1]: #same line
                if (seq1[i][0]==seq1[i-1][0]+1 and seq2[i][0]==seq2[i-1][0]-1) or \
                (seq1[i][0]==seq1[i-1][0]-1 and seq2[i][0]==seq2[i-1][0]+1) :
                    flag = 1
                else:
                    flag = 0
            elif seq1[i][0] == seq1[i-1][0]: #same line
                if (seq1[i][1]==seq1[i-1][1]-1 and seq2[i][1]==seq2[i-1][1]-1) or \
                (seq1[i][1]==seq1[i-1][1]+1 and seq2[i][1]==seq2[i-1][1]+1) :
                    flag = 1 
                else:
                    flag = 0
            else:
                flag = 0
    return flag 

In [43]:
seq3 = [(4,2),(4,3),(5,3),(5,2),(6,2),(6,1),(7,1)] 
seq4 = [(4,2),(4,3),(3,3),(3,2),(2,2),(2,1),(2,0)]
seq5 = [(4,2),(4,3),(3,3),(3,2),(2,2),(2,1),(1,1)]
print(are_symmetrical(seq4,seq3))
print(are_symmetrical(seq5,seq3))

0
1


In [44]:
def branch_and_bound_addition(l ,k ): #k = profondeur et n = hauteur ie profondeur max
    best_score_inter = 0 
    new_sol= []
    if k == len(string_of_H_and_P):
        best_score_inter = score(l, string_of_H_and_P)[0]
        new_sol = l
    else:
        for dir in ['left', 'middle' , 'right']:
            new_index = checking(l, dir) #return None si occupé, index vide si vide
            if new_index != None:
                l2 = copy.copy(l)
                l2.append(new_index)
                a,b = branch_and_bound_addition(l2, k+1)
                if a >= best_score_inter:
                    best_score_inter = a
                    new_sol = b
                    
    return best_score_inter, new_sol 

In [45]:
string_of_H_and_P = "HPPHHH"
print('string_of_H_and_P =', string_of_H_and_P)
n = len(string_of_H_and_P)
list_of_two_first_elements = two_first_elements(n)
print('two first elements=',list_of_two_first_elements)
start = datetime.now()
scoring, sol = branch_and_bound_addition(list_of_two_first_elements, 2)
end = datetime.now()
elapsed = end - start
formatted_computation_time = float('%i.%.6i' % (elapsed.seconds, elapsed.microseconds))
print('computation time =',formatted_computation_time, 'seconds')
print("sol=", sol)
print("score=", scoring)
print(display(sol,string_of_H_and_P))

string_of_H_and_P = HPPHHH
two first elements= [(4, 2), (4, 3)]
computation time = 0.004025 seconds
sol= [(4, 2), (4, 3), (5, 3), (5, 2), (5, 1), (4, 1)]
score= 2
[[" (6, 'H') " " (1, 'H') " " (2, 'P') "]
 [" (5, 'H') " " (4, 'H') " " (3, 'P') "]]


Now, we will test the algorothm with the provided sequences : HHHHHHH , HPPHPHP, HHHPHPPHPHPPHHHPH

In [46]:
string_of_H_and_P = "HHHHHHH"
print('string_of_H_and_P =', string_of_H_and_P)
n = len(string_of_H_and_P)
list_of_two_first_elements = two_first_elements(n)
print('two first elements=',list_of_two_first_elements)
start = datetime.now()
scoring, sol = branch_and_bound_addition(list_of_two_first_elements, 2)
end = datetime.now()
elapsed = end - start
formatted_computation_time = float('%i.%.6i' % (elapsed.seconds, elapsed.microseconds))
print('computation time =',formatted_computation_time, 'seconds')
print("sol=", sol)
print("score=", scoring)
print(display(sol,string_of_H_and_P))

string_of_H_and_P = HHHHHHH
two first elements= [(5, 3), (5, 4)]
computation time = 0.02224 seconds
sol= [(5, 3), (5, 4), (6, 4), (6, 3), (6, 2), (5, 2), (4, 2)]
score= 2
[[" (7, 'H') " '          ' '          ']
 [" (6, 'H') " " (1, 'H') " " (2, 'H') "]
 [" (5, 'H') " " (4, 'H') " " (3, 'H') "]]


In [29]:
string_of_H_and_P = "HPPHPHP"
print('string_of_H_and_P =', string_of_H_and_P)
n = len(string_of_H_and_P)
list_of_two_first_elements = two_first_elements(n)
print('two first elements=',list_of_two_first_elements)
start = datetime.now()
scoring, sol = branch_and_bound_addition(list_of_two_first_elements, 2)
end = datetime.now()
elapsed = end - start
formatted_computation_time = float('%i.%.6i' % (elapsed.seconds, elapsed.microseconds))
print('computation time =',formatted_computation_time, 'seconds')
print("sol=", sol)
print("score=", scoring)
print(display(sol,string_of_H_and_P))

string_of_H_and_P = HPPHPHP
two first elements= [(5, 3), (5, 4)]
computation time = 0.006709 seconds
sol= [(5, 3), (5, 4), (6, 4), (6, 3), (6, 2), (5, 2), (4, 2)]
score= 2
[[" (7, 'P') " '          ' '          ']
 [" (6, 'H') " " (1, 'H') " " (2, 'P') "]
 [" (5, 'P') " " (4, 'H') " " (3, 'P') "]]


In [30]:
string_of_H_and_P = "HHHPHPPHPHPPHHHPH"
print('string_of_H_and_P =', string_of_H_and_P)
n = len(string_of_H_and_P)
list_of_two_first_elements = two_first_elements(n)
print('two first elements=',list_of_two_first_elements)
start = datetime.now()
scoring, sol = branch_and_bound_addition(list_of_two_first_elements, 2)
end = datetime.now()
elapsed = end - start
formatted_computation_time = float('%i.%.6i' % (elapsed.seconds, elapsed.microseconds))
print('computation time =',formatted_computation_time, 'seconds')
print("sol=", sol)
print("score=", scoring)
print(display(sol,string_of_H_and_P))

string_of_H_and_P = HHHPHPPHPHPPHHHPH
two first elements= [(15, 13), (15, 14)]
computation time = 323.740001 seconds
sol= [(15, 13), (15, 14), (15, 15), (16, 15), (16, 14), (17, 14), (17, 13), (16, 13), (16, 12), (15, 12), (15, 11), (14, 11), (14, 12), (14, 13), (14, 14), (13, 14), (13, 13)]
score= 8
[['          ' '          ' " (17, 'H')" " (16, 'P')" '          ']
 [" (12, 'P')" " (13, 'H')" " (14, 'H')" " (15, 'H')" '          ']
 [" (11, 'P')" " (10, 'H')" " (1, 'H') " " (2, 'H') " " (3, 'H') "]
 ['          ' " (9, 'P') " " (8, 'H') " " (5, 'H') " " (4, 'P') "]
 ['          ' '          ' " (7, 'P') " " (6, 'P') " '          ']]
